# TensorFlow

**[CNN-01]** 必要なモジュールをインポートして、乱数のシードを設定します。

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

np.random.seed(20160704)
tf.set_random_seed(20160704)

/Users/yohei/.pyenv/versions/anaconda3-5.0.0/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


**[CNN-02]** MNISTのデータセットを用意します。

In [2]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


**[CNN-03]** 1段目の畳み込みフィルターとプーリング層を定義します。

In [3]:
num_filters1 = 32

x = tf.placeholder(tf.float32, [None, 784])
x_image = tf.reshape(x, [-1,28,28,1])  # NHWC

# He の初期値の利用
he_init = tf.contrib.layers.variance_scaling_initializer(
            mode='FAN_IN',
            uniform=False,
            seed=None,
            dtype=tf.float32
        )

W_conv1 = tf.Variable(he_init([5,5,1,num_filters1]))  # FH,FW,C,FN
# 畳み込み
h_conv1 = tf.nn.conv2d(x_image, W_conv1,
                       strides=[1,1,1,1], padding='SAME')
b_conv1 = tf.Variable(tf.constant(0.1, shape=[num_filters1]))  # FNのshape  0.1の定数で初期化

# ReLUを適用
h_conv1_cutoff = tf.nn.relu(h_conv1 + b_conv1)  # h_conv1 + b_conv1 : ブロードキャストを適用

h_pool1 = tf.nn.max_pool(h_conv1_cutoff, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

**[CNN-04]** 2段目の畳み込みフィルターとプーリング層を定義します。

In [4]:
num_filters2 = 64

W_conv2 = tf.Variable(he_init([5,5,num_filters1,num_filters2]))
# 畳み込み
h_conv2 = tf.nn.conv2d(h_pool1, W_conv2,
                       strides=[1,1,1,1], padding='SAME')

b_conv2 = tf.Variable(tf.constant(0.1, shape=[num_filters2]))
# ReLUを適用
h_conv2_cutoff = tf.nn.relu(h_conv2 + b_conv2)

h_pool2 = tf.nn.max_pool(h_conv2_cutoff, ksize=[1,2,2,1],
                         strides=[1,2,2,1], padding='SAME')

**[CNN-05]** 全結合層、ドロップアウト層、ソフトマックス関数を定義します。

In [5]:
# 全結合層に渡すためにflatにする
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*num_filters2])

num_units1 = 7*7*num_filters2
num_units2 = 1024

# 全結合
w2 = tf.Variable(he_init([num_units1, num_units2]))
b2 = tf.Variable(tf.constant(0.1, shape=[num_units2]))
# ReLUを適用
hidden2 = tf.nn.relu(tf.matmul(h_pool2_flat, w2) + b2)

# ドロップアウト層  keep_probは訓練中とテスト中で値を変えるのでplaceholderにする
keep_prob = tf.placeholder(tf.float32)
hidden2_drop = tf.nn.dropout(hidden2, keep_prob=keep_prob)  # keep_probはドロップアウトしない確率を意味する

w0 = tf.Variable(tf.zeros([num_units2, 10]))
b0 = tf.Variable(tf.zeros([10]))
logits = tf.matmul(hidden2_drop, w0) + b0
p = tf.nn.softmax(logits)

**[CNN-06]** 誤差関数 loss、トレーニングアルゴリズム train_step、正解率 accuracy を定義します。

In [6]:
t = tf.placeholder(tf.float32, [None, 10])
loss = -tf.reduce_mean(tf.reduce_sum(t * tf.log(p), axis=1))
"""
上の式と同様
loss =  tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                                                     labels=t)
one_hot出ない場合はこちら
labels = tf.argmax(t, axis=1)
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                                                                                                    labels=labels)
"""
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)
correct_prediction = tf.equal(tf.argmax(p, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

**[CNN-07]** セッションを用意して、Variable を初期化します。

In [7]:
sess = tf.Session()
sess.run(tf.initialize_all_variables())
saver = tf.train.Saver()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


**[CNN-08]** パラメーターの最適化を20000回繰り返します。

最終的に、テストセットに対して約99%の正解率が得られます。

In [10]:
i = 0
for _ in range(20000):
    i += 1
    batch_xs, batch_ts = mnist.train.next_batch(50)
    sess.run(train_step,
             feed_dict={x:batch_xs, t:batch_ts, keep_prob:0.5})   # 学習にはkeep_prob:0.5を設定
    if i % 500 == 0:
        loss_vals, acc_vals = [], []
        for c in range(4):
            start = len(mnist.test.labels) / 4 * c
            end = len(mnist.test.labels) / 4 * (c+1)
            start, end = int(start), int(end)
            loss_val, acc_val = sess.run([loss, accuracy],
                feed_dict={x:mnist.test.images[start:end],
                           t:mnist.test.labels[start:end],
                           keep_prob:1.0})  # テスト時にはkeep_prob:1.0を設定
            loss_vals.append(loss_val)
            acc_vals.append(acc_val)
        loss_val = np.sum(loss_vals)
        acc_val = np.mean(acc_vals)
        print ('Step: %d, Loss: %f, Accuracy: %f'
               % (i, loss_val, acc_val))
        saver.save(sess, 'tensorflow_session/cnn_session', global_step=i)

Step: 500, Loss: 1065.137451, Accuracy: 0.968700
Step: 1000, Loss: 773.044617, Accuracy: 0.974100
Step: 1500, Loss: 602.194519, Accuracy: 0.981000
Step: 2000, Loss: 522.263672, Accuracy: 0.983000
Step: 2500, Loss: 434.533264, Accuracy: 0.985600
Step: 3000, Loss: 405.385529, Accuracy: 0.985700
Step: 3500, Loss: 384.260925, Accuracy: 0.986700
Step: 4000, Loss: 341.971405, Accuracy: 0.988400
Step: 4500, Loss: 306.159027, Accuracy: 0.989700
Step: 5000, Loss: 349.742798, Accuracy: 0.988200
Step: 5500, Loss: 282.624084, Accuracy: 0.990900
Step: 6000, Loss: 265.764435, Accuracy: 0.990600
Step: 6500, Loss: 263.285767, Accuracy: 0.990800
Step: 7000, Loss: 264.778595, Accuracy: 0.990200
Step: 7500, Loss: 302.817505, Accuracy: 0.990100
Step: 8000, Loss: 252.953705, Accuracy: 0.991000
Step: 8500, Loss: 238.789688, Accuracy: 0.991200
Step: 9000, Loss: 283.603302, Accuracy: 0.991000
Step: 9500, Loss: 232.585037, Accuracy: 0.991600
Step: 10000, Loss: 234.978195, Accuracy: 0.991300
Step: 10500, Loss: 

In [9]:
start

0.0

**[CNN-09]** セッション情報を保存したファイルが生成されていることを確認します。

In [12]:
!ls tensorflow_session/cnn_session*

tensorflow_session/cnn_session-18000.data-00000-of-00001
tensorflow_session/cnn_session-18000.index
tensorflow_session/cnn_session-18000.meta
tensorflow_session/cnn_session-18500.data-00000-of-00001
tensorflow_session/cnn_session-18500.index
tensorflow_session/cnn_session-18500.meta
tensorflow_session/cnn_session-19000.data-00000-of-00001
tensorflow_session/cnn_session-19000.index
tensorflow_session/cnn_session-19000.meta
tensorflow_session/cnn_session-19500.data-00000-of-00001
tensorflow_session/cnn_session-19500.index
tensorflow_session/cnn_session-19500.meta
tensorflow_session/cnn_session-20000.data-00000-of-00001
tensorflow_session/cnn_session-20000.index
tensorflow_session/cnn_session-20000.meta


In [13]:
sess.close()

# Keras

In [18]:
from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.initializers import he_normal, Constant

In [32]:
np.random.seed(20160703)

In [35]:
model = Sequential()

In [36]:
num_filters1, num_filters2 = 32, 64, 
num_units1, num_units2 = 7*7*num_filters2, 1024

with K.name_scope('CNN'):
    
    # 畳み込み1
    model.add(Conv2D(filters=num_filters1, kernel_size=5,
                     padding="same",
                     data_format="channels_last",
                     input_shape=(28, 28, 1),  # 第一層に使うときのみinput_shape引数が必要になる
                    kernel_initializer=he_normal(),
                     bias_initializer=Constant(value=0.1),
                    name='convolution1',
                     use_bias=True
                    ))
    with K.name_scope('pooling1'): # TensorFlowのname_scopeが使える． 色々とTensorFlowと変換性がある
        model.add(MaxPooling2D(pool_size=(2, 2),
                           padding="same", 
                           data_format="channels_last",
                            name='pool1'))
        
    # 畳み込み2
    model.add(Conv2D(filters=num_filters2, kernel_size=5,
                     padding="same",
                     data_format="channels_last",
                    kernel_initializer=he_normal(),
                      bias_initializer=Constant(value=0.1),
                    name='convolution2',
                     use_bias=True
                    ))
    with K.name_scope('pooling2'): # TensorFlowのname_scopeが使える． 色々とTensorFlowと変換性がある
        model.add(MaxPooling2D(pool_size=(2, 2),
                           padding="same", 
                           data_format="channels_last",
                            name='pool2'))
        
        model.add(Flatten())  #  入力を平滑化する．バッチサイズに影響されない．

    model.add(Dense(units=num_units2, activation='relu', name='fully-connected'))
    model.add(Dropout(rate=0.5))  # TFと違い，rate引数にはドロップアウトする確率を渡す．
    model.add(Dense(units=10, activation='softmax', name='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
convolution1 (Conv2D)        (None, 28, 28, 32)        832       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 14, 14, 32)        0         
_________________________________________________________________
convolution2 (Conv2D)        (None, 14, 14, 64)        51264     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 3136)              0         
_________________________________________________________________
fully-connected (Dense)      (None, 1024)              3212288   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
__________

In [38]:
tensorboad = TensorBoard(log_dir='/tmp/mnist_df_logs', histogram_freq=1)  # histogram_freq=50で 50 epochごとに記録することになる
earlystopping = EarlyStopping(monitor='val_loss',  patience=5, verbose=1)

# ModelCheckpointでエポックごとにセッションを保存 : 詳しくは「詳解ディープラーニング  p300~」
checkpoint = ModelCheckpoint(
                            filepath='keras_session/model_{epoch:02d}_vloss{val_loss:.3f}.hdf5',
                            save_best_only=True)  #  save_best_only=True　だと そのとき最もよかった結果を上書きしないで済む?

In [39]:
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=0.0005),
             metrics=['accuracy'])

In [40]:
x_train, t_train = mnist.train.images,mnist.train.labels
x_train.shape, t_train.shape

((55000, 784), (55000, 10))

In [41]:
x_test, t_test = mnist.test.images,mnist.test.labels
x_test.shape, t_test.shape

((10000, 784), (10000, 10))

In [42]:
!mkdir keras_session  # Kerasの場合はディレクトリを作らないといけない

In [43]:
# 学習
x_train = x_train.reshape([-1,28,28,1])
x_test = x_test.reshape([-1,28,28,1])

hist = model.fit(x_train, t_train, batch_size=50,
                         epochs=19,
                         validation_data=(x_test, t_test),
                         callbacks=[checkpoint, earlystopping])

Train on 55000 samples, validate on 10000 samples
Epoch 1/19
55000/55000 [==============================] - 207s 4ms/step - loss: 0.1565 - acc: 0.9520 - val_loss: 0.0447 - val_acc: 0.9857
Epoch 2/19
55000/55000 [==============================] - 205s 4ms/step - loss: 0.0572 - acc: 0.9817 - val_loss: 0.0609 - val_acc: 0.9809
Epoch 3/19
55000/55000 [==============================] - 200s 4ms/step - loss: 0.0441 - acc: 0.9863 - val_loss: 0.0437 - val_acc: 0.9874
Epoch 4/19
55000/55000 [==============================] - 204s 4ms/step - loss: 0.0352 - acc: 0.9886 - val_loss: 0.0506 - val_acc: 0.9853
Epoch 5/19
55000/55000 [==============================] - 203s 4ms/step - loss: 0.0316 - acc: 0.9900 - val_loss: 0.0396 - val_acc: 0.9889
Epoch 6/19
55000/55000 [==============================] - 214s 4ms/step - loss: 0.0277 - acc: 0.9913 - val_loss: 0.0376 - val_acc: 0.9901
Epoch 7/19
55000/55000 [==============================] - 203s 4ms/step - loss: 0.0245 - acc: 0.9925 - val_loss: 0.0517 - 

In [44]:
1100

1100

# テスト

In [55]:
sess = tf.InteractiveSession()

In [70]:
logits = tf.constant([[1., -3, 10]])
labels = tf.constant([[0, 0, 1]])

xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                                                                  labels=labels)
xentropy.eval()

array([0.00012564], dtype=float32)

In [65]:
logits = tf.constant([1., -3, 10])
labels = tf.constant([0.1, 0.02, 0.88])

xentropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                                                                  labels=labels)
xentropy.eval()

1.1601256

In [88]:
logits = tf.constant([[1., -3, 10]])
labels = tf.constant([2])

xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,
                                                                                                          labels=labels)
xentropy.eval()

array([0.00012564], dtype=float32)

In [86]:
logits = tf.constant([[1., -3., 10.]])
labels = tf.constant([[0., 0., 1.]])

p = tf.nn.softmax(logits)

loss = -tf.reduce_mean(tf.reduce_sum( labels* tf.log(p), axis=1))
loss.eval()

0.00012565448